In [ ]:
# Getting data from aerospace database.
using CSV, DataFrames, Statistics, Random
include("../test/load.jl")

In [ ]:
# Optional extra cores
using Distributed
addprocs(5)

In [ ]:
columns = ["Re", "thick", "M", "C_L"];
X = CSV.read(OCT.DATA_DIR * "airfoil/airfoil_X.csv", DataFrame, copycols=true, header=columns, delim=",");
Y = CSV.read(OCT.DATA_DIR * "airfoil/airfoil_Y.csv", DataFrame, copycols=true, header=["C_D"], delim=",");
X = exp.(X); Y = exp.(Y);
Re = Array(range(10000, stop=35000, step=5000));
thick = [100,110,120,130,140,145];
M = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9];
cl = Array(range(0.35, stop=0.70, step=0.05));

In [ ]:
# Plotting some perspectives on the data
using Plots
plt = Plots.plot(X[:,1], X[:,4], Y[:,1], seriestype=:scatter, markersize = 2)
display(plt)

In [ ]:
# Splitting data
(train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, seed=1, X, Y, train_proportion=0.8);
size(train_X)

In [ ]:
# Let's do fun things with the data.
lnr = IAI.OptimalTreeRegressor(random_seed=1, max_depth=3, cp=0.001, minbucket=0.05, regression_sparsity=:all, 
    regression_lambda = 0.0001)
IAI.fit!(lnr, train_X, train_y[:,1])

In [ ]:
# Cross-validation of best tree
grids = IAI.GridSearch(lnr, max_depth = [3, 4], minbucket=[0.03,0.06,0.09])
IAI.fit!(grids, train_X, train_y[:,1], test_X, test_y[:,1])

In [ ]:
# Seeing best learner
lnr = IAI.get_learner(grids)

In [ ]:
# How about trying sparse hyperplane splits? 
lnr = IAI.OptimalTreeRegressor(random_seed=1, max_depth=3, cp=0.001, minbucket=0.05, 
    hyperplane_config=(sparsity=2,), regression_sparsity=:all, regression_lambda = 0.0001, fast_num_support_restarts =10)
grids = IAI.GridSearch(lnr, max_depth = [2,3], minbucket=[0.03,0.06,0.09,0.12,0.15])
IAI.fit!(grids, train_X, train_y[:,1], test_X, test_y[:,1])

In [ ]:
# Seeing best learner
lnr = IAI.get_learner(grids)

In [ ]:
# MSE errors 
println("Training MSE:", 1 - IAI.score(lnr, train_X, train_y[:,1], criterion=:mse))
println("Test MSE:", 1- IAI.score(lnr, test_X, test_y[:,1], criterion=:mse))
# MSE error of global posynomial
Re = test_X[:,1]; thickness = test_X[:,2]; M = test_X[:,3]; C_L = test_X[:,4]; C_D = test_y[:,1];
CDp = 0.0470226 .* (Re).^-0.388166 .* thickness.^0.782129 .* (M).^-0.339824 .* (C_L).^0.94829 +
    190.63 .* (Re).^-0.218175 .* thickness.^3.94137 .* (M).^19.2346 .* (C_L).^1.14997 +
    1.62158 .* (Re).^-0.549562 .* thickness.^1.2895 .* (M).^3.03057 .* (C_L).^1.77464 +
    2.91642e-12 .* (Re).^1.18062 .* thickness.^-1.75547 .* (M).^0.105431 .*(C_L).^-1.4407;
CDp = CDp.^(1/1.64722);
MSEposy = mean((C_D-CDp).^2);
println("Test MSE of global posynomial: ", MSEposy)

In [ ]:
# Plotting some perspectives on the data, with predictions
using Plots
all_leaves = find_leaves(lnr)
leaf_index = IAI.apply(lnr, X);
predictions = IAI.predict(lnr, X)
# plt = Plots.plot(X[:,1], X[:,4], Y[:,1], seriestype=:scatter, markersize = 2)
plt = Plots.plot(X[:,1], X[:,4], predictions, zcolor=leaf_index, seriestype=:scatter, markersize = 2)
display(plt)